# Settings Components

> UI components for workflow settings modal and forms

In [ ]:
#| default_exp settings.components

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from fasthtml.common import *

from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, items, justify
from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_tailwind.utilities.interactivity import cursor
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_interactions.patterns.modal_dialog import ModalDialog, ModalTriggerButton, ModalSize

## settings_trigger_button

Button that triggers the settings modal.

In [ ]:
#| export
def settings_trigger_button(
    modal_id:str,                  # ID of the modal to trigger
    label:str="Settings",          # Button label text
    button_cls:Optional[str]=None  # Optional additional button classes
) -> FT:                           # Button element that triggers the modal
    """Create a button that opens the settings modal."""
    cls = combine_classes(btn, btn_styles.ghost, btn_sizes.sm)
    if button_cls:
        cls = combine_classes(cls, button_cls)

    return ModalTriggerButton(
        modal_id=modal_id,
        label=label,
        button_cls=str(cls)
    )

## simple_settings_form

Generate a simple settings form from a JSON schema.

In [ ]:
#| export
def simple_settings_form(
    directories:list,        # List of media directories
    auto_save:bool,          # Current auto-save setting
    results_directory:str,   # Current results directory
    save_url:str,            # URL to POST settings to
    target_id:str,           # Target element ID for HTMX response
    modal_id:str             # Modal ID for close button
) -> FT:                     # Simple form element
    """Create a simple settings form without full schema generation."""
    from cjm_fasthtml_daisyui.components.data_input.checkbox import checkbox
    from cjm_fasthtml_daisyui.components.data_input.text_input import text_input

    return Form(
        # Directories (simplified as textarea)
        Div(
            Label("Media Directories (one per line):", cls=str(font_weight.semibold)),
            Textarea(
                "\n".join(directories),
                name="directories",
                rows="4",
                cls=combine_classes("textarea", "textarea-bordered", w.full, m.t(1))
            ),
            cls=str(m.b(4))
        ),

        # Auto-save toggle
        Div(
            Label(
                Input(
                    type="checkbox",
                    name="auto_save",
                    checked=auto_save,
                    cls=str(checkbox)
                ),
                Span("Auto-save transcription results", cls=str(m.l(2))),
                cls=combine_classes(flex_display, items.center, cursor.pointer)
            ),
            cls=str(m.b(4))
        ),

        # Results directory
        Div(
            Label("Results Directory:", cls=str(font_weight.semibold)),
            Input(
                type="text",
                name="results_directory",
                value=results_directory,
                cls=combine_classes(text_input, w.full, m.t(1))
            ),
            cls=str(m.b(4))
        ),

        # Action buttons
        Div(
            Button(
                "Cancel",
                type="button",
                onclick=f"document.getElementById('modal-dialog-{modal_id}').close()",
                cls=combine_classes(btn, btn_styles.ghost)
            ),
            Button(
                "Save",
                type="submit",
                cls=combine_classes(btn, btn_colors.primary)
            ),
            cls=combine_classes(flex_display, gap(2), justify.end, m.t(6))
        ),

        hx_post=save_url,
        hx_target=f"#{target_id}",
        hx_swap="innerHTML"
    )

## settings_modal

Settings modal dialog with tabbed sections for different setting categories.

In [ ]:
#| export
def settings_modal(
    modal_id:str,                    # ID for the modal element
    schema:Dict[str, Any],           # JSON schema for settings
    current_values:Dict[str, Any],   # Current settings values
    save_url:str,                    # URL to POST settings to
    target_id:str                    # Target element ID for HTMX response
) -> FT:                             # Modal dialog with settings form
    """Create the settings modal with form."""
    from cjm_fasthtml_jsonschema.generators.form import generate_form_ui

    # Generate form UI from schema
    form_content = generate_form_ui(
        schema=schema,
        values=current_values,
        show_title=False,  # We'll add our own title
        show_description=True,
        compact=True,
        card_wrapper=False
    )

    # Get the correct dialog ID using the helper
    dialog_id = InteractionHtmlIds.modal_dialog(modal_id)

    modal_content = Div(
        # Title
        H3(
            schema.get("title", "Settings"),
            cls=combine_classes(font_size.xl, font_weight.bold, m.b(4))
        ),

        # Form with schema-generated fields
        Form(
            form_content,

            # Action buttons
            Div(
                Button(
                    "Cancel",
                    type="button",
                    onclick=f"document.getElementById('{dialog_id}').close()",
                    cls=combine_classes(btn, btn_styles.ghost)
                ),
                Button(
                    "Save",
                    type="submit",
                    cls=combine_classes(btn, btn_colors.primary)
                ),
                cls=combine_classes(flex_display, gap(2), justify.end, m.t(6))
            ),

            hx_post=save_url,
            hx_target=f"#{target_id}",
            hx_swap="innerHTML"
        ),

        cls=str(p(4))
    )

    # Use auto_show=True so modal opens when loaded via HTMX
    return ModalDialog(
        modal_id=modal_id,
        content=modal_content,
        size=ModalSize.LARGE,
        show_close_button=True,
        close_on_backdrop=True,
        auto_show=True
    )

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()